In [ ]:
import os
import re
from glob import glob
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# find dat file within a run
def find_dat_in_run(run_root):
    dats = []
    #dats = glob(os.path.join(run_root,"Outputs/*.dat"))
    for proot, dirs, files in os.walk(run_root):
        for f in files:
            if f.find('_trunc.dat') > -1:
                dats.append(os.path.join(run_root,proot,f))
    return dats

In [ ]:
runs_root = "C:/JSPRuns"
comp_dirs = ["2025-03-04-154054_AMMC_5019_NewBase",
             "2025-03-04-155000_AMMC_5100_Base_3x80ocFLTrains",
             "2025-03-06-114718_AMMC_5101_Base_3x80ocFLTrn_FL_MW_Sidings",
             "4TrainSweep/4Trains/2024-09-09-161136_AMMC_3403_FL4x80oc_LoadUnloadUp_FM",
             "2025-03-05-153736_AMMC_5102_Base_3x80ocFLTrn_Quebec_pls2",
            ]

dats = [find_dat_in_run(os.path.join(runs_root, d))[0] for d in comp_dirs]
dats

In [ ]:
dfs = []
for dat_file in dats:
    print(dat_file)
    df = pd.read_csv(dat_file,delimiter="\t")
    df = df.loc[:,~df.columns.str.match(r"^Unnamed")]
    df.loc[:,"run"] = os.path.splitext(os.path.split(dat_file)[-1])[0]
    dfs.append(df)

In [ ]:
df = pd.concat(dfs).reset_index(drop=True)

def date_or_nan(x):
    try:
        return datetime.strptime(x[0],"%Y-%m-%d-%H%M%S")
    except ValueError:
        return pd.NaT
        
#df["run_date"] = df["run"].str.split("_").apply(date_or_nan)
df["run_id"] = df["run"].str.extract(r"(\d{4})_")
#df

## Total shipped from Fire Lake

In [ ]:
grp_col = "run_id"
val_col = "[cvMWUnloadFL].TotalIn([Ore])/1[Mt]"

box_plot = sns.boxplot(data=df, x=grp_col,y=val_col,)
lbls = [x.get_text() for x in box_plot.get_xticklabels()]

means = df.groupby(grp_col)[val_col].mean()[lbls]
vertical_offset = df[val_col].mean() * 0.02 # offset from median for displayvertical_offset = df[val_col].mean() * 0.02 # offset from median for display

for xtick in box_plot.get_xticks():
    box_plot.text(xtick,means[xtick] + vertical_offset,round(means[xtick],ndigits=1), 
            horizontalalignment='center',color='k')
plt.ylabel("Unloaded by FL (Mtpa)")
plt.grid(True)

## Total shipped at port

In [ ]:
val_col = "sum([cvToMarine].TotalIn)/1[Mt]"

box_plot = sns.boxplot(data=df, x=grp_col,y=val_col,)
lbls = [x.get_text() for x in box_plot.get_xticklabels()]

means = df.groupby(grp_col)[val_col].mean()[lbls]
vertical_offset = df[val_col].mean() * 0.02 # offset from median for display

for xtick in box_plot.get_xticks():
    box_plot.text(xtick,means[xtick] + vertical_offset,round(means[xtick],ndigits=1), 
            horizontalalignment='center',color='k')
plt.ylabel("Total Shipped (Mtpa)")
plt.grid(True)

## Train cycle time

In [ ]:
def read_log(fn,columns=None):
    lno = find_empty_line_no(fn)
    print(lno, fn)
    start_row = lno+1
    if columns is not None:
        start_row += 1
    df = pd.read_csv(fn,delimiter="\t",skiprows=start_row,names=columns)
    df = df.loc[:,~df.columns.str.match(r"^Unnamed")]
    return df

def get_fl_cycle_logs_in_run(run_root):
    logs = []
    for proot, dirs, files in os.walk(run_root):
        for f in files:
            if f.find('CyclesFL.log') > -1:
                logs.append(os.path.join(run_root,proot,f))
    return logs

def cycle_log_from_log(df):
    first_State = df.iloc[0].Loc_State
    df_cycle = df[df.Loc_State == first_State]
    df_cycle.loc[df_cycle.iloc[:-1].index,"CycleTime"] = df_cycle.Time.diff()
    return df_cycle


train_log_cols = ["Time", "Loc_State", "Func_State"]

def get_fl_cycle_logs_in_run(run_root,warmup_time=0):
    logs = []
    for proot, dirs, files in os.walk(run_root):
        for f in files:
            if re.match(r".*exLogTrainFL\d+State\.log", f):
                tno = re.match(r".*(\d+)State\.log", f).groups()[0]
                log = read_log(os.path.join(proot,f),columns=train_log_cols)
                cycle_log = cycle_log_from_log(log)
                cycle_log["Train"] = int(tno)
                logs.append(cycle_log)
    return pd.concat(logs)

def find_empty_line_no(f):
    lno=0
    with open(f,"r") as fid:
        for line in fid:
            if len(line) < 2:
                return lno
            lno += 1

In [ ]:
#fl_log_files = [get_fl_cycle_logs_in_run(os.path.join(runs_root, x))[0] for x in comp_dirs]
fl_logs=[]

warmup_time = 24*28

for d in comp_dirs:
    
    dfs = get_fl_cycle_logs_in_run(os.path.join(runs_root, d))
    
    fl_logs.append(dfs)

In [ ]:
fl_logs

In [ ]:
l=fl_logs[0]

In [ ]:
s = l.iloc[0].Loc_State

In [ ]:
l[l.Loc_State==s].Time.diff().describe()

In [ ]:
len(l[l.Loc_State==s])